In [11]:
%load_ext autoreload
%autoreload 2
from glob import glob
import gc
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.ml_utils import save_feature, get_cnt_feature, get_dummie_feature, get_label_feature
from func.time_utils import date_add_days, diff_of_days, diff_of_times
from kaggle_utils import reduce_mem_usage, move_feature
from ieee_utils import get_os_release_date

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]
base_train = read_pkl_gzip('../input/base_train.gz').set_index(COLUMN_ID)
base_test = read_pkl_gzip('../input/base_test.gz').set_index(COLUMN_ID)
base = pd.concat([base_train, base_test], axis=0)

train_iden = pd.read_csv(f'../input/train_identity.csv', index_col='TransactionID')
test_iden = pd.read_csv(f'../input/test_identity.csv', index_col='TransactionID')

In [37]:
# START_DATE = '2017-11-04'
START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
base['datetime'] = base['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
base['date'] = base['datetime'].map(lambda x: x.date())

In [50]:
def trans_DT(df):
    list_regist = []
    for d, diff in tqdm(df[['date', 'D1']].values):
        if diff < 999999:
            regist = date_add_days(d, -1*diff)
        else:
            regist = date_add_days(d, 0)
        list_regist.append(str(regist))
    df['Regist_date'] = list_regist
    return df
base = trans_DT(base)

100%|██████████| 1097231/1097231 [00:04<00:00, 273167.24it/s]


In [13]:
def id_split(df):
    df['DeviceInfo'].fillna('#', inplace=True)
    df['DeviceInfo'] = df['DeviceInfo'].map(lambda x: x.lower())
    
    df['device_name'] = df['DeviceInfo'].str.split('/', expand=True)[0]
    df['device_version'] = df['DeviceInfo'].str.split('/', expand=True)[1]
    df['device_country'] = 'Unk'

    df['id_29'].fillna('#', inplace=True)
    df['id_30'].fillna('#', inplace=True)
    df['id_29'] = df['id_29'].map(lambda x: x.lower())
    df['id_30'] = df['id_30'].map(lambda x: x.lower().replace('_', '.'))
    df['id_30'] = df['id_30'].map(lambda x: x.replace('mac os x', 'mac') if x.count('mac') else x)
    df['OS_id_30'] = df['id_30'].str.split(' ', expand=True)[0]
    df['version_id_30'] = df['id_30'].map(lambda x: x.split(' ')[1] if x.count(' ') else '0' )

    df['screen_width'] = df['id_33'].str.split('x', expand=True)[0]
    df['screen_height'] = df['id_33'].str.split('x', expand=True)[1]

    df['id_34'] = df['id_34'].str.split(':', expand=True)[1]
    df['id_23'] = df['id_23'].str.split(':', expand=True)[1]

    df.loc[df['device_name'].str.contains('sm', na=False), 'device_name'] = 'samsung'
    df.loc[df['device_name'].str.contains('samsung', na=False), 'device_name'] = 'samsung'
    df.loc[df['device_name'].str.contains('gt-', na=False), 'device_name'] = 'samsung'
    df.loc[df['device_name'].str.contains('moto g', na=False), 'device_name'] = 'motorola'
    df.loc[df['device_name'].str.contains('moto', na=False), 'device_name'] = 'motorola'
    df.loc[df['device_name'].str.contains('moto', na=False), 'device_name'] = 'motorola'
    df.loc[df['device_name'].str.contains('lg-', na=False), 'device_name'] = 'lg'
    df.loc[df['device_name'].str.contains('rv:', na=False), 'device_name'] = 'rv'
    df.loc[df['device_name'].str.contains('huawei', na=False), 'device_name'] = 'huawei'
    df.loc[df['device_name'].str.contains('blade', na=False), 'device_name'] = 'zte'
    df.loc[df['device_name'].str.contains('blade', na=False), 'device_name'] = 'zte'
    df.loc[df['device_name'].str.contains('linux', na=False), 'device_name'] = 'linux'
    df.loc[df['device_name'].str.contains('xt', na=False), 'device_name'] = 'sony'
    df.loc[df['device_name'].str.contains('htc', na=False), 'device_name'] = 'htc'
    df.loc[df['device_name'].str.contains('asus', na=False), 'device_name'] = 'asus'
    df.loc[df['device_name'].str.contains('hi', na=False), 'device_name'] = 'huawei'
    df.loc[df['device_name'].str.contains('ale-', na=False), 'device_name'] = 'huawei'
    df.loc[df['device_name'].str.contains('-l', na=False), 'device_name'] = 'huawei'
    # Add
    df.loc[df['device_name'].str.contains('ios device', na=False), 'device_name'] = 'ios'
    df.loc[df['device_name'].str.contains('build', na=False), 'DeviceInfo']   = 'build'
    
    df['device_country'] = df['DeviceInfo'].map(lambda x: 
                                                'Korea' if x.count('sm')
                                                else 'Korea' if x.count('samsung')
                                                else 'Korea' if x.count('gt-'   )
                                                else 'China' if x.count('moto g')
                                                else 'China' if x.count('moto'  )
                                                else 'China' if x.count('moto'  )
                                                else 'Korea' if x.count('lg'    )
                                                else 'RV' if x.count('rv:'      )
                                                else 'China' if x.count('hi'    )
                                                else 'China' if x.count('redmi' )
                                                else 'China' if x.count('huawei')
                                                else 'China' if x.count('ale-'  )
                                                else 'China' if x.count('-l'    )
                                                else 'China' if x.count('blade' )
                                                else 'China' if x.count('blade' )
                                                else 'US' if x.count('linux'    )
                                                else 'Japan' if x.count('xt'    )
                                                else 'China' if x.count('htc'   )
                                                else 'China' if x.count('asus'  )
                                                else 'Build' if x.count('build' )
                                                else 'US' if x.count('window'   )
                                                else 'US' if x.count('ios'      )
                                                else 'US' if x.count('mac'      )
                                                else 'China' if x.count('lenovo')
                                                else 'US' if x.count('pixel'    )
                                                else 'Unk'
                                               )
    
    df.loc[df.device_name.isin(df.device_name.value_counts()[df.device_name.value_counts() < 200].index), 'device_name'] = "others"
    df['device_name'].fillna('#', inplace=True)
    df['device_version'].fillna('0', inplace=True)
    df['fixed_DeviceInfo'] = df[['device_name', 'device_version']].apply(lambda x: x[0] + '__' + x[1], axis=1)
    gc.collect()
    
    return df

In [18]:
data = pd.concat([train_iden, test_iden], axis=0)
data = id_split(data)

In [19]:
#========================================================================
# OS Date
#========================================================================
os_map = get_os_release_date()
o_list = []
v_list = []
list_os_date = []
data['OS_id_30'] = data['OS_id_30'].map(lambda x: x.lower())
data['version_id_30'] = data['version_id_30'].map(lambda x: x.lower())

for os, ver in data[['OS_id_30', 'version_id_30']].values:
    date = np.nan
    if os in os_map:
        ver_map = os_map[os]
        ver = ver.replace('_', '.').replace(' ', '')
        
        if os.count('window'):
            if ver in ver_map:
                date = ver_map[ver]
            elif ver.count('x'):
                date = ver_map['xp']
            elif ver.count('v'):
                date = ver_map['vista']
            elif ver.count('7'):
                date = ver_map['7']
            elif ver.count('8'):
                date = ver_map['8']
            elif ver.count('8') and ver.count('1'):
                date = ver_map['8.1']
            elif ver.count('10'):
                date = ver_map['10']
            else:
                if ver!='0':
                    print(ver)
            
        elif os.count('mac'):
            if ver in ver_map:
                date = ver_map[ver]
            elif len(ver)==5:
                d_ver = ver[:2] + '.' + ver[2:4] + '.' + ver[4]
                if d_ver in ver_map:
                    date = ver_map[d_ver]
                else:
                    print(ver)
            elif len(ver)==4:
                d_ver = ver[:2] + '.' + ver[2:]
                d3_ver = ver[:2] + '.' + ver[2] + '.' + ver[3]
                if d_ver in ver_map:
                    date = ver_map[d_ver]
                elif d3_ver in ver_map:
                    date = ver_map[d3_ver]
                else:
                    print(ver)
            elif len(ver)==3:
                d_ver = ver[:2] + '.' + ver[2]
                if d_ver in ver_map:
                    date = ver_map[d_ver]
                else:
                    print(ver)
            else:
                if ver!='0':
                    ov.append(os + ver)
                
        elif os.count('ios'):
            if ver in ver_map:
                date = ver_map[ver]
            elif len(ver)==5:
                d_ver = ver[:2] + '.' + ver[2:4] + '.' + ver[4]
                if d_ver in ver_map:
                    date = ver_map[d_ver]
                else:
                    print(ver)
            elif len(ver)==4:
                d3_ver = ver[:2] + '.' + ver[2] + '.' + ver[3]
                if d3_ver in ver_map:
                    date = ver_map[d3_ver]
                else:
                    print(ver)
            elif len(ver)==3:
                d3_ver = ver[0] + '.' + ver[1] + '.' + ver[2]
                if d3_ver in ver_map:
                    date = ver_map[d3_ver]
                else:
                    print(ver)
            else:
                if ver!='0':
                    print(ver)
            
        elif os.count('andro'):
            if ver in ver_map:
                date = ver_map[ver]
            elif ver.replace('.', '_') in ver_map:
                date = ver_map[ver.replace('.', '_')]
            elif len(ver)==5:
                print(ver)
            elif len(ver)==4:
                print(ver)
            elif len(ver)==3:
                d3_ver = ver[0] + '.' + ver[1] + '.' + ver[2]
                if d3_ver in ver_map:
                    date = ver_map[d3_ver]
                else:
                    print(ver)
            else:
                if ver!='0':
                    print(ver)
        else:
            print(os, ver)
    else:
        o_list.append(os)
        v_list.append(ver)
            
    list_os_date.append(date)
print(set(o_list))
print(set(v_list))
data['os_release_date'] = list_os_date

{'#', 'func', 'linux', 'other'}
{'0'}


In [ ]:
#========================================================================
# Browser Date
#========================================================================
brow_date = pd.read_csv('../input/0903_ieee__browser_release_date.csv').set_index('browser')['ReleaseDate']
data['browser_release_date'] = data['id_31'].map(brow_date)

In [75]:
# User id を付与し、最初の日付との差分をとったりする
df_same = pd.read_csv('../output/same_user_pattern/0902__same_user_id__card_addr_pemail_M.csv').set_index(COLUMN_ID)
base['predicted_user_id'] = df_same['predicted_user_id']
data_dt = data.join(base, how='left')

user_min_dt = data_dt.groupby('predicted_user_id')['datetime'].min()
user_max_dt = data_dt.groupby('predicted_user_id')['datetime'].max()
data_dt['first_datetime'] = data_dt['predicted_user_id'].map(user_min_dt)
data_dt['last_datetime'] = data_dt['predicted_user_id'].map(user_max_dt)

In [199]:
#========================================================================
# FE OS Browser Date
#========================================================================

data_dt['os_release_date'] = pd.to_datetime( data_dt['os_release_date'] )

data_dt['diff__TransactionDT-os_release_date'] = data_dt['datetime'] - data_dt['os_release_date']
data_dt['diff__TransactionDT-os_release_date'] = data_dt['diff__TransactionDT-os_release_date'].map(lambda x: x.days)

data_dt['Regist_date'] = pd.to_datetime(data_dt['Regist_date'])
data_dt['diff__os_release_date-Regist_date'] = data_dt['Regist_date'] - data_dt['os_release_date']
data_dt['diff__os_release_date-Regist_date'] = data_dt['diff__os_release_date-Regist_date'].map(lambda x: x.days)

data_dt['diff__os_release_date-first_TransactionDT'] = data_dt['first_datetime'] - pd.to_datetime(data_dt['os_release_date'])
data_dt['diff__os_release_date-first_TransactionDT'] = data_dt['diff__os_release_date-first_TransactionDT'].map(lambda x: x.days)

data_dt['diff__os_release_date-last_TransactionDT'] = data_dt['last_datetime'] - pd.to_datetime(data_dt['os_release_date'])
data_dt['diff__os_release_date-last_TransactionDT'] = data_dt['diff__os_release_date-last_TransactionDT'].map(lambda x: x.days)


data_dt['browser_release_date'] = pd.to_datetime( data_dt['browser_release_date'] )

data_dt['diff__TransactionDT-browser_release_date'] = data_dt['datetime'] - data_dt['browser_release_date']
data_dt['diff__TransactionDT-browser_release_date'] = data_dt['diff__TransactionDT-browser_release_date'].map(lambda x: x.days)

data_dt['Regist_date'] = pd.to_datetime(data_dt['Regist_date'])
data_dt['diff__browser_release_date-Regist_date'] = data_dt['Regist_date'] - data_dt['browser_release_date']
data_dt['diff__browser_release_date-Regist_date'] = data_dt['diff__browser_release_date-Regist_date'].map(lambda x: x.days)

data_dt['diff__browser_release_date-first_TransactionDT'] = data_dt['first_datetime'] - pd.to_datetime(data_dt['browser_release_date'])
data_dt['diff__browser_release_date-first_TransactionDT'] = data_dt['diff__browser_release_date-first_TransactionDT'].map(lambda x: x.days)

data_dt['diff__browser_release_date-last_TransactionDT'] = data_dt['last_datetime'] - pd.to_datetime(data_dt['browser_release_date'])
data_dt['diff__browser_release_date-last_TransactionDT'] = data_dt['diff__browser_release_date-last_TransactionDT'].map(lambda x: x.days)

data_dt['diff__os_release_date-browser_release_date'] = data_dt['os_release_date'] - data_dt['browser_release_date']
data_dt['diff__os_release_date-browser_release_date'] = data_dt['diff__os_release_date-browser_release_date'].map(lambda x: x.days)

In [234]:
#========================================================================
# OS
#========================================================================
data_dt['id_31'].fillna('#', inplace=True)
data_dt['device_os'] = data_dt[['device_name', 'id_31']].apply(lambda x: 
                                              'windows' if x[0].count('window')
                                              else 'mac' if x[0].count('mac')
                                              else 'ios' if x[0].count('ios')
                                              else 'android' if x[0].count('samsun')
                                              else 'windows' if x[0].count('trid')
                                              else 'android' if x[0].count('moto')
                                              else 'android' if x[0].count('huaw')
                                              else 'android' if x[0].count('lg')
                                              else 'android' if x[0].count('sony')
                                              else 'android' if x[0].count('htc')
                                              else 'android' if x[0].count('zte')
                                              else 'linux' if x[0].count('linux')
                                              else 'windows' if x[0].count('rv')
                                              else 'android' if x[0].count('#') and x[1].count('android')
                                              else 'android' if x[0].count('#') and x[1].count('google')
                                              else 'android' if x[0].count('#') and x[1].count('sam')
                                              else 'ios' if x[0].count('#') and x[1].count('mobile')
                                              else 'mac' if x[0].count('#') and x[1].count('safari')
                                              else 'windows' if x[0].count('#') and x[1].count('chrome')
                                              else 'windows' if x[0].count('#') and x[1].count('opera')
                                              else 'windows' if x[0].count('#') and x[1].count('ie')
                                              else 'windows' if x[0].count('#') and x[1].count('edge')
                                              else 'windows' if x[0].count('#') and x[1].count('desktop')
                                              else 'android' if x[0].count('#') and x[1].count('fire')
                                              else 'android' if x[0].count('#') and x[1].count('line')
                                                       
                                              else 'android' if x[0].count('ther') and x[1].count('android')
                                              else 'android' if x[0].count('ther') and x[1].count('google')
                                              else 'android' if x[0].count('ther') and x[1].count('sam')
                                              else 'ios' if x[0].count('ther') and x[1].count('mobile')
                                              else 'mac' if x[0].count('ther') and x[1].count('safari')
                                              else 'windows' if x[0].count('ther') and x[1].count('chrome')
                                              else 'windows' if x[0].count('ther') and x[1].count('opera')
                                              else 'windows' if x[0].count('ther') and x[1].count('ie')
                                              else 'windows' if x[0].count('ther') and x[1].count('edge')
                                              else 'windows' if x[0].count('ther') and x[1].count('desktop')
                                              else 'windows' if x[0].count('ther') and x[1].count('fire')
                                              else 'android' if x[0].count('ther') and x[1].count('line')
                                              else 'other_device_os'
                                             , axis=1)

windows    133842
android     58872
ios         56271
mac         26923
other        9928
linux         304
Name: os_type, dtype: int64

In [187]:
data_dt['browser_name'] = data_dt['id_31'].map(lambda x: 
                     'generic' if x.count('generic')
                     else 'safari' if x.count('mobile')
                     else 'safari' if x.count('safari')
                     else 'chrome' if x.count('chrome')
                     else 'firefox' if x.count('firefox')
                     else 'ie' if x.count('edge')
                     else 'ie' if x.count('ie')
                     else 'ie' if x.count('window')
                     else 'android' if x.count('android')
                     else 'desktop' if x.count('desktop')
                     else 'ios' if x.count('ios')
                     else 'samusung' if x.count('samsun')
                     else 'google' if x.count('google')
                     else 'opera' if x.count('opera')
                     else 'other_browser_name'
                    )

data_dt['browser_os'] = data_dt['id_31'].map(lambda x: 
                     'ios' if x.count('mobile')
                     else 'mac' if x.count('safari')
                     else 'android' if x.count('chrome')
                     else 'windows' if x.count('firefox')
                     else 'windows' if x.count('edge')
                     else 'windows' if x.count('ie')
                     else 'windows' if x.count('window')
                     else 'android' if x.count('android')
                     else 'windows' if x.count('desktop')
                     else 'ios' if x.count('ios')
                     else 'android' if x.count('samsun')
                     else 'android' if x.count('google')
                     else 'other_browser_os'
                    )

chrome 63.0            28428
safari 11.0            27102
chrome 70.0            24343
safari generic         19669
safari 12.0            16648
chrome 71.0            14939
chrome 69.0            14620
ie 11.0                14203
chrome 65.0            10883
chrome 64.0            10618
chrome 68.0            10278
chrome 67.0            10013
chrome 62.0             9604
#                       9233
chrome 66.0             7725
chrome generic          5936
safari 10.0             4991
edge 17.0               4945
edge 16.0               4592
firefox 57.0            3331
firefox 63.0            1954
edge 15.0               1725
firefox 61.0            1423
samsung browser 7.4     1369
ie 11.0 for tablet      1330
chrome 61.0             1320
firefox 64.0            1251
safari 9.0              1235
firefox 62.0            1173
firefox 59.0            1152
                       ...  
safari                    36
Microsoft/Windows         25
silk                      19
mobile        

In [ ]:
#========================================================================
# FE OS Match Device and Browser
#========================================================================
data_dt['os_match_device_browser'] = (data_dt['device_os'] == data_dt['browser_os'])*1

In [142]:
#========================================================================
# FE Categorical Encoding 
#========================================================================

cols_categorical = get_categorical_features(data, ignore_list=COLUMNS_IGNORE)
df_cat = data[cols_categorical]
for col in cols_categorical:
    num = df_cat[col].value_counts().shape[0]
    df_cat[col].fillna('#', inplace=True)
    cols_cat = [col]
    if num>15:
        cnt_feature = get_cnt_feature(df_cat[col].to_frame(), cols_cat)
        label_feature = get_label_feature(df_cat[col].to_frame(), cols_cat)
        df_cat = df_cat.join(cnt_feature).join(label_feature)
    elif num>2:
        cnt_feature = get_cnt_feature(df_cat[col].to_frame(), cols_cat)
        label_feature = get_label_feature(df_cat[col].to_frame(), cols_cat)
        dummie_feature = get_dummie_feature(df_cat[col].to_frame(), cols_cat)
        df_cat = df_cat.join(cnt_feature).join(label_feature).join(dummie_feature)
    elif num<=2:
        label_feature = get_label_feature(df_cat[col].to_frame(), cols_cat)
        df_cat = df_cat.join(label_feature)
    df_cat.drop(col, axis=1, inplace=True)

In [146]:
base = pd.concat([base_train, base_test], axis=0)
base = base.join(df_cat, how='left')

In [108]:
dir_save = 'org_use'
save_feature(base_train, '501__', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
save_feature(base_test, '501__', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(590540,) | id_01
(590540,) | id_02
(590540,) | id_03
(590540,) | id_04
(590540,) | id_05
(590540,) | id_06
(590540,) | id_07
(590540,) | id_08
(590540,) | id_09
(590540,) | id_10
(590540,) | id_11


ValueError: could not convert string to float: 'NotFound'